In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_selection import VarianceThreshold# Input data files are available in the read-only "../input/" directory
import keras
from keras.models import Sequential
from keras.optimizers import SGD
from keras import layers 
from keras.layers import Activation, Dense ,Dropout, BatchNormalization, Input,LeakyReLU
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss
from sklearn import preprocessing
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import seaborn as sns
import matplotlib.pyplot as plt
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Mechanisms of Action (MoA) Prediction

The goal of the challenge is to classify drugs based on their biological activity to identify certain proteins that are associated with a specific disease and then develop molecules that can target those proteins.

The data set describes the responses of 100 different types of human cells to various drugs. These response patterns will be used to classify the response of the MoA.

It is a multi-tag classification problem. Drugs can have multiple MoA annotations that describe binary responses from different cell types in different ways. The evaluation metric is the mean log loss per column.

The data comes in the familiar form of test and training files. Unlike other competitions, here we have two separate files for training predictors (train_features.csv) and targets (train_targets_scored.csv). Each row corresponds to a specific treatment.

## 1. Loading the dataset

In [ ]:
train=pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_target=pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test=pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
target_cols=train_target.columns[1:]
target_cols

In [ ]:
test_preds.loc[:, target_cols]

In [ ]:
print('train.shape:', train.shape)
print('train_target.shape:', train_target.shape)
print('test.shape:', test.shape)
print('submission.shape:', submission.shape)

In [ ]:
train['cp_type'].value_counts()

In [ ]:
train['cp_time'].value_counts()

In [ ]:
train['cp_dose'].value_counts()

The training and test sets include 876 coulmns.
- sig_id: The unique id of the dataset.
- cp_type: Catigorical feature which represents samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs.
- cp_time: Catigorical feature which indicates treatment duration (24, 48, 72 hours). 	 	
- cp_dose: Catigorical feature which represents high or low dose.
- g-: Continous features which signify gene expression data.
- c-: Continous features which signify cell viability data.

## 2. Features Engineering and EDA

### 2.1. Target engineering

In [ ]:
train_target.head()
print ("Shape of train target",train_target.shape)

The target are dummy encoded which will allow the computation of the predicted probability for each class. in Multi-label classification problem we predict variables and our target variables here are 206.

In [ ]:
ax = train_target.drop('sig_id',axis=1)\
.sum() \
.sort_values(ascending=False)\
.head(30)\
.sort_values()\
.plot(kind='barh',
     figsize=(15, 10)
     )
ax.set_title('Top 30 Scored Targets Classification Counts', fontsize=20)
plt.show

### 2.2. Handeling Missing Values

In [ ]:
print('Missing values in training dataset:', train.isnull().sum().sum())
print('Missing values in testing dataset:', test.isnull().sum().sum())
print('Missing values in target dataset:', train_target.isnull().sum().sum())

There are no missing values in the datasets

### 2.3 Handeling Categorical Features

As we mentioned above there are three catigrical features.
- cp_type
- cp_time
- cp_dose

We will handel them using *Label encoding* approach. 


* **Why are all labels zero where cp_type = ctrl_vehicle ie for control?**

Cells set to zero because they are control/samples without compounds (no MoA). So, we are going to drop all rows of the vehicle treatment.

In [ ]:
p_train=train.copy()
p_test=test.copy()
p_train_target=train_target.copy()
#p_train.drop(p_train[p_train['cp_type']=='ctl_vehicle'].index, inplace = True)
#p_test.drop(p_test[p_test['cp_type']=='ctl_vehicle'].index, inplace = True)
p_train['cp_time'] = p_train['cp_time'].map( {24: 1, 48: 2, 72: 3} ).astype(int)
p_test['cp_time'] = p_test['cp_time'].map( {24: 1, 48: 2, 72: 3} ).astype(int)
p_train['cp_type'] = p_train['cp_type'].map( {'trt_cp': 1, 'ctl_vehicle': 2} ).astype(int)
p_test['cp_type'] = p_test['cp_type'].map( {'trt_cp': 1, 'ctl_vehicle': 2} ).astype(int)
p_train['cp_dose'] = p_train['cp_dose'].map( {'D1': 1, 'D2': 2} ).astype(int)
p_test['cp_dose'] = p_test['cp_dose'].map( {'D1': 1, 'D2': 2} ).astype(int)
p_train.drop('sig_id',axis='columns', inplace=True)
p_test.drop('sig_id',axis='columns', inplace=True)
p_train_target.drop('sig_id',axis='columns', inplace=True)

Now, we'll preprocess training and testing datasets using MinMaxScaler to scale each feature individually.

In [ ]:
# Fit scaler to joint train and test data
#scaler = preprocessing.MinMaxScaler()
#scaler.fit(p_train.append(p_test))

#train_trans = scaler.transform(p_train)
#test_trans = scaler.transform(p_test)

#p_train = pd.DataFrame(train_trans, columns=p_train.columns)
#p_test = pd.DataFrame(test_trans, columns=p_test.columns)

### 2.3 Numeric feature engineering


There are 872 numeric features devided to two categories; features represent the genes and features represent the cells.

- **Why are there 23k samples when there are only 5k drugs?**

    A perturbagen ,which is a method of providing information about the operation of pathways and networks within a cell, is applied to a mixture of cell lines not to a single cell. also, one control could be used for many different drugs and even other drugs could be also used as controls in specific cases.


#### **Interpreting Gene Expression Features**

Gene expression features are 772 and represent the mRNA level or in other words, the process of synthesizing a protein.

Gene features values are giving an insight whether a drug has an effect on the gene or no, where high values >2 or <-2 says there is a measurable effect on the gene unlike values close to zero.

In [ ]:
g_features = [cols for cols in train.columns if cols.startswith('g-')]

In [ ]:
color = ['dimgray','navy','purple','orangered', 'red', 'green' ,'mediumorchid', 'khaki', 'salmon', 'blue','cornflowerblue','mediumseagreen']
 
color_ind=0
n_row = 6
n_col = 3
n_sub = 1 
plt.rcParams["legend.loc"] = 'upper right'
fig = plt.figure(figsize=(8,14))
plt.subplots_adjust(left=-0.3, right=1.3,bottom=-0.3,top=1.3)
for i in (np.arange(0,6,1)):
    plt.subplot(n_row, n_col, n_sub)
    sns.kdeplot(train.loc[:,g_features[i]],color=color[color_ind],shade=True,
                 label=['mean:'+str('{:.2f}'.format(train.loc[:,g_features[i]].mean()))
                        +'  ''std: '+str('{:.2f}'.format(train.loc[:,g_features[i]].std()))])
    
    plt.xlabel(g_features[i])
    plt.legend()                    
    n_sub+=1
    color_ind+=1
plt.show()

- **What is the cell viability features?**

cell viability assays use multi markers to indicat active healthy cells in a population. also to optimize experimental conditions following treatment.

and we have 100 cell-viability features (c-0 to c-99). Each cell-viability feature represents viability of one particular cell.

In [ ]:
c_features = [cols for cols in train.columns if cols.startswith('c-')]

In [ ]:
n_row = 6
n_col = 3
n_sub = 1 
fig = plt.figure(figsize=(8,14))
plt.subplots_adjust(left=-0.3, right=1.3,bottom=-0.3,top=1.3)
plt.rcParams["legend.loc"] = 'upper left'
for i in (np.arange(0,6,1)):
    plt.subplot(n_row, n_col, n_sub)
    sns.kdeplot(train.loc[:,c_features[i]],color=color[color_ind],shade=True,
                 label=['mean:'+str('{:.2f}'.format(train.loc[:,c_features[i]].mean()))
                        +'  ''std: '+str('{:.2f}'.format(train.loc[:,c_features[i]].std()))])
    
    plt.xlabel(c_features[i])
    plt.legend()                    
    n_sub+=1
    color_ind+=1
plt.show()

## 3. NN Model

In [ ]:
y_train = p_train_target
x_train = p_train
x_test = p_test
n_cols = x_train.shape[1]

In [ ]:
# Hypermeters
SEED = 1234
EPOCHS = 28
BATCH_SIZE = 128
FOLDS = 5
REPEATS = 5
LR = 0.0005
target_cols=train_target.columns[1:]

### 3.1 Cross Validation and Keras Model Definition

In [ ]:
def train(resume_models = None, repeat_number = 0, folds = 5, skip_folds = 0):
    
    models = []
    prediction = y_train.copy()
    
    # Enumarating in 5 folds
    kfold = KFold(folds, shuffle = True)
    # Splitting the training set to train and validaation set
    for fold, (train_inx, v_inx) in enumerate(kfold.split(x_train)):
        print('\n')
        print('-'*50)
        print(f'Repeat:{repeat_number} Training Fold:{fold+1}')
        
        # Checkpointing with ReduceLROnPlateau to reduce learning rate then save the best model only
        cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'binary_crossentropy', factor = 0.4, patience = 2, verbose = 1, min_delta = 0.0001, mode = 'auto')
        checkpoint_path = f'repeat:{repeat_number}_Fold:{fold}.hdf5'
        cb_checkpt = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True, save_weights_only = True, mode = 'min')
        
        # Build the NN model
        model = Sequential()
        #three input layers with relu activation function
        model.add(Dense(2048, input_dim=n_cols, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(2048,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(206, activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])        
        #fit the model
        model.fit(x_train.values[train_inx],y_train.values[train_inx],validation_data=(x_train.values[v_inx], y_train.values[v_inx]),callbacks = [cb_lr_schedule, cb_checkpt],epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2)
        
        #load the checkpoimts
        model.load_weights(checkpoint_path)
        prediction.loc[v_inx, :] = model.predict(x_train.values[v_inx])
        models.append(model)

    return models, prediction

### 3.2 Model Training

In [ ]:
models = []
prediction = []
# reproducability
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
tf.random.set_seed(SEED)

#Repeat
for i in range(REPEATS):
    m, oof = train(repeat_number = i, folds=FOLDS)
    models = models + m
    prediction.append(oof)

### 3.3 Test Predictions and Save Submission


Submissions are scored by the log loss:

$\Large \text{log loss} = - \frac{1}{M}\sum_{m=1}^{M} \frac{1}{N} \sum_{i=1}^{N} \left[ y_{i,m} \log(\hat{y}_{i,m}) + (1 - y_{i,m}) \log(1 - \hat{y}_{i,m})\right]$

* $N$ is the number of rows ($i=1,…,N$)
* $M$ is the number of targets ($m=1,…,M$)
* $\large \hat{y}_{i,m}$ is the predicted probability of the ith row and mth target
* $\large y_{i,m}$ is the ground truth of the ith row and mth target (1 for a positive response, 0 otherwise)
* $log()$ is the natural logarithm

In [ ]:
def multi_log_loss(y_actual, y_pred):
    y_pred = np.clip(y_pred, 1e-15, (1 - 1e-15))
    score = - np.mean(np.mean(y_actual * np.log(y_pred) + (1 - y_actual) * np.log(1 - y_pred), axis=1))
    return score

In [ ]:
mean_preds = y_train.copy()
mean_preds.loc[:, target_cols] = 0
for i, pred in enumerate(prediction):
    print(f'Repeat {i + 1} Log Loss: {multi_log_loss(y_train, pred)}')
    mean_preds.loc[:, target_cols] += pred[target_cols]

mean_preds.loc[:, target_cols] /= len(prediction)
print(f'Mean Log Loss: {multi_log_loss(y_train, mean_preds)}')
mean_preds.loc[x_train['cp_type'] == 0, target_cols] = 0
print(f"Mean Log Loss (ctl adjusted): {multi_log_loss(y_train, mean_preds)}")

In [ ]:
test_preds = submission.copy()
test_preds[ target_cols] = 0
for model in models:
    test_preds.loc[:, target_cols] += model.predict(x_test)
test_preds.loc[:, target_cols] /= len(models)
test_preds.loc[x_test['cp_type'] == 0,  target_cols] = 0
test_preds.to_csv('submission.csv', index=False)